In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import BCELoss
# import torchvision.transforms as transforms
from torchvision.transforms import v2 as transforms
import torchvision.datasets as datasets
import torchvision.models as models
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split

In [2]:
# Define NTXentLoss (provided by you)
class NTXentLoss(nn.Module):
    def __init__(self, temperature=0.5):
        super(NTXentLoss, self).__init__()
        self.temperature = temperature

    def forward(self, z_i, z_j):
        batch_size = z_i.size(0)
        z = torch.cat([z_i, z_j], dim=0)
        z = F.normalize(z, dim=1)
        similarity_matrix = torch.mm(z, z.T) / self.temperature
        mask = torch.eye(2 * batch_size, device=z.device).bool()
        similarity_matrix.masked_fill_(mask, -float('inf'))
        exp_sim = torch.exp(similarity_matrix)
        denominator = exp_sim.sum(dim=1)
        positive_samples = torch.cat(
            [torch.arange(batch_size, 2 * batch_size), torch.arange(batch_size)], dim=0
        ).to(z.device)
        positives = similarity_matrix[torch.arange(2 * batch_size), positive_samples]
        loss = -torch.log(torch.exp(positives) / denominator)
        return loss.mean()

In [3]:
# Load and preprocess the MNIST dataset
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
])

mnist_data = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_data, eval_data = train_test_split(mnist_data, train_size=0.1, random_state=42, stratify=mnist_data.targets)

# Bag-level dataset
class BagDataset(Dataset):
    def __init__(self, data, bag_size=8):
        self.data = data
        self.bag_size = bag_size

    def __len__(self):
        return len(self.data) // self.bag_size

    def __getitem__(self, idx):
        images, labels = [], []
        for i in range(self.bag_size):
            img, label = self.data[idx * self.bag_size + i]
            images.append(img)
            labels.append(label)
        bag_label = 1 if 9 in labels else 0
        return torch.stack(images), torch.tensor(bag_label)

train_dataset = BagDataset(train_data)
eval_dataset = BagDataset(eval_data)
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, pin_memory=True, num_workers=4, drop_last=True)
eval_loader = DataLoader(eval_dataset, batch_size=128, shuffle=False, pin_memory=True, num_workers=4, drop_last=True)

/home/hskha23/miniconda3/envs/tf/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


In [4]:
class Attention(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(Attention, self).__init__()
        self.attention = nn.Sequential(
            nn.Linear(in_dim, 256),
            nn.ReLU(),
            nn.Linear(256, out_dim)
        )

    def forward(self, x):
        attention_weights = self.attention(x)
        weights = F.softmax(attention_weights, dim=1)
        return (x * weights).sum(dim=1), weights.squeeze(-1)

In [5]:
class Encoder(nn.Module):
    def __init__(self, base_model, projection_dim=128):
        super(Encoder, self).__init__()
        self.encoder = base_model
        self.projection = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, projection_dim)
        )
        
        self.attention = Attention(512, 1)
        # self.fc = nn.Linear(512, 1)
        self.fc = nn.Sequential(
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )
    def forward(self, x):
        batch_size, num_instances, channels, height, width = x.size()
        x = x.view(-1, channels, height, width)  # Reshape to (batch_size * num_instances, channels, height, width)
        features = self.encoder(x)
        features = nn.Dropout(0.25)(features)
        features = features.view(batch_size * num_instances, -1)  # Flatten to (batch_size * num_instances, feature_dim)
        
        projection_features = self.projection(features)
        attention_features, _ = self.attention(features.view(batch_size, num_instances, -1))
        output = self.fc(attention_features)
        
        return projection_features, output

# Augmentation function
def augment_batch(batch_images):
    batch_size, num_instances, channels, height, width = batch_images.shape
    aug_transform = transforms.Compose([
        # transforms.RandomResizedCrop(224, scale=(0.8, 1.1)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomApply([transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3, hue=0.3)], p=0.6),
        transforms.RandomGrayscale(p=0.2),
        transforms.ToTensor(),
        # transforms.Normalize((0.1307,), (0.3081,))
    ])
    
    # Apply transformation to each image instance in the batch
    augmented_batch = []
    for i in range(batch_size):
        augmented_instances = [aug_transform(transforms.ToPILImage()(img.cpu())) for img in batch_images[i]]
        augmented_batch.append(torch.stack(augmented_instances))
    
    return torch.stack(augmented_batch).cuda()  # Move the augmented batch to GPU

# Initialize models, loss function, and optimizer
base_model = models.resnet18(weights=None)
base_model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
base_model.fc = nn.Identity()
encoder = Encoder(base_model).cuda()
projection_dim = 256
ntxent_loss = NTXentLoss().cuda()

In [6]:
# Training parameters
epochs = 100
learning_rate = 1e-3
optimizer = torch.optim.Adam(encoder.parameters(), lr=learning_rate)
bceLoss = nn.BCEWithLogitsLoss()


correct_predictions = 0
total_samples = 0
# Training loop for contrastive learning
for epoch in range(epochs):
    encoder.train()
    total_loss = 0
    for images, labels in train_loader:
        images = images.cuda()  # Move the batch to GPU
        labels = labels.cuda()
        # aug1 = augment_batch(images).cuda()
        aug1 = images 
        aug2 = augment_batch(images).cuda()
        
        z_i, outputs_1 = encoder(aug1)
        z_j, outputs_2 = encoder(aug2)
        
        NTXLoss = ntxent_loss(z_i, z_j)
        BCELoss_1 = bceLoss(outputs_1.squeeze(), labels.float())
        BCELoss_2 = bceLoss(outputs_2.squeeze(), labels.float())
        loss = 0.4 * NTXLoss + 0.3 * BCELoss_1 + 0.3 * BCELoss_2
        loss = loss.mean()
        # loss = ntxent_loss(z_i, z_j)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
        # Calculate predictions and update correct predictions count
        predicted = (torch.sigmoid(outputs_1.squeeze()) > 0.5).float()  # Binary classification threshold
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    # Calculate average loss and accuracy for this epoch
    avg_loss = total_loss / len(train_loader)
    accuracy = (correct_predictions / total_samples) * 100  # Convert to percentage
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss/len(train_loader):.5f}, Accuracy: {accuracy:.2f}%")
    
# Evaluation loop
with torch.no_grad():
    encoder.eval()
    correct, total = 0, 0
    for images, labels in eval_loader:
        images = images.cuda()
        labels = labels.cuda()
        z_i, output = encoder(images)
        predicted = (torch.sigmoid(output.squeeze()) > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print(f"Test Accuracy: {correct/total}")

/home/hskha23/miniconda3/envs/tf/lib/python3.10/site-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


Epoch [1/100], Loss: 3.39113, Accuracy: 59.22%
Epoch [2/100], Loss: 3.02312, Accuracy: 66.09%
Epoch [3/100], Loss: 2.88068, Accuracy: 73.02%
Epoch [4/100], Loss: 2.78280, Accuracy: 78.09%
Epoch [5/100], Loss: 2.74355, Accuracy: 81.78%
Epoch [6/100], Loss: 2.68787, Accuracy: 84.58%
Epoch [7/100], Loss: 2.65900, Accuracy: 86.63%
Epoch [8/100], Loss: 2.62379, Accuracy: 88.16%
Epoch [9/100], Loss: 2.61141, Accuracy: 89.32%
Epoch [10/100], Loss: 2.58888, Accuracy: 90.27%
Epoch [11/100], Loss: 2.58131, Accuracy: 91.09%
Epoch [12/100], Loss: 2.55641, Accuracy: 91.76%
Epoch [13/100], Loss: 2.54758, Accuracy: 92.31%
Epoch [14/100], Loss: 2.53070, Accuracy: 92.81%
Epoch [15/100], Loss: 2.51705, Accuracy: 93.29%
Epoch [16/100], Loss: 2.49763, Accuracy: 93.70%
Epoch [17/100], Loss: 2.48363, Accuracy: 94.05%
Epoch [18/100], Loss: 2.48901, Accuracy: 94.36%
Epoch [19/100], Loss: 2.47312, Accuracy: 94.62%
Epoch [20/100], Loss: 2.46989, Accuracy: 94.88%
Epoch [21/100], Loss: 2.47246, Accuracy: 95.09%
E

## Freeze the encoder and train a linear classifier

In [7]:
# # Freeze the encoder
# # Function to freeze all layers of a model
# def freeze_model(model):
#     for param in model.parameters():
#         param.requires_grad = False
# 
# # Freeze the contrastive learning model
# freeze_model(encoder)

## Define the linear classifier

In [8]:
# class LinearClassifier(nn.Module):
#     def __init__(self, in_dim, out_dim):
#         super(LinearClassifier, self).__init__()
#         self.fc = nn.Sequential(
#             nn.Linear(in_dim, 128),
#             nn.ReLU(),
#             nn.Linear(128, out_dim)
#         )
#         self.dropout = nn.Dropout(0.25)
#         self.attention = Attention(in_dim, 1)
# 
#     def forward(self, x):
#         x = x.view(32, 8, 128)
#         # # Max pooling over the bag instances
#         # x, _ = torch.max(x, dim=1)
#         x = self.dropout(x)
#         x, _ = self.attention(x)
#         return self.fc(x)

In [9]:
# # Initialize the linear classifier
# classifier = LinearClassifier(in_dim=projection_dim, out_dim=1).cuda()
# 
# # Training parameters
# epochs = 50
# learning_rate = 5e-4
# optimizer = torch.optim.Adam(classifier.parameters(), lr=learning_rate)
# criterion = nn.BCEWithLogitsLoss()

In [10]:
# # Training loop for linear classification with accuracy calculation
# for epoch in range(epochs):
#     classifier.train()
#     total_loss = 0
#     correct_predictions = 0
#     total_samples = 0
#     
#     for images, labels in train_loader:
#         images, labels = images.cuda(), labels.float().cuda()
#         features = encoder(images)
#         outputs = classifier(features).squeeze()
#         
#         # Calculate the loss
#         loss = criterion(outputs, labels)
#         
#         # Backpropagation and optimization
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         
#         total_loss += loss.item()
# 
#         # Calculate predictions and update correct predictions count
#         predicted = (torch.sigmoid(outputs) > 0.5).float()  # Binary classification threshold
#         correct_predictions += (predicted == labels).sum().item()
#         total_samples += labels.size(0)
# 
#     # Calculate average loss and accuracy for this epoch
#     avg_loss = total_loss / len(train_loader)
#     accuracy = (correct_predictions / total_samples) * 100  # Convert to percentage
# 
#     print(f"Epoch [{epoch + 1}/{epochs}], Loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%")

In [11]:
# # Evaluation loop
# classifier.eval()
# encoder.eval()
# correct, total = 0, 0
# with torch.no_grad():
#     for images, labels in eval_loader:
#         images, labels = images.cuda(), labels.float().cuda()
#         # images = augment_batch(images)
#         features = encoder(images)
#         outputs = classifier(features).squeeze()
#         predicted = (torch.sigmoid(outputs) > 0.5).float()
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()
# print(f"Accuracy: {correct/total}")
# 
# # Save the model
# torch.save(encoder.state_dict(), "encoder.pth")
# torch.save(classifier.state_dict(), "classifier.pth")